## Linear data to be consumed by a neural network

build a classification network and perform binary classification on the result data

In [13]:
import os
import glob

from tqdm import tqdm
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
from torch.nn import functional as F

## 1. Load data

load data and do some explorative data analysis

- Todo normalize the data (use a rescaler from scikit)
- Which features do have which values

In [14]:
def moveTo(obj, device): 
    """ 
    obj: the python object to move to a device, or to move its
    ➥ contents to a device
    device: the compute device to move objects to 
    """
    if isinstance(obj, list): 
        return [moveTo(x, device) for x in obj] 
    elif isinstance(obj, tuple): 
        return tuple(moveTo(list(obj), device)) 
    elif isinstance(obj, set): 
        return set(moveTo(list(obj), device)) 
    elif isinstance(obj, dict): 
        to_ret = dict() 
        for key, value in obj.items(): 
            to_ret[moveTo(key, device)] = moveTo(value, device) 
        return to_ret 
    elif hasattr(obj, "to"): 
        return obj.to(device) 
    else: 
        return obj

In [15]:
dataset_path = r'C:\data\wsc'
df_filepaths = glob.glob(os.path.join(dataset_path, '**/*.csv'), recursive=True)

In [16]:
def extract_data_from_csv(csv_filepath: str):
    df = pd.read_csv(df_filepaths[0], index_col=False)
    df = df.drop(columns=['Unnamed: 0'])
    x = df.drop(columns=['is_ws']).to_numpy()
    y = df['is_ws'].to_numpy()
    return x, y

In [17]:
x_data = []
y_data = []

for df_path in df_filepaths:
    x, y = extract_data_from_csv(df_path)
    x_data.append(x)
    y_data.append(y)

x_data = np.concatenate(x_data)
y_data = np.concatenate(y_data)

## 2. Modeling

In [18]:
class WSDataset(Dataset):
  def __init__(self,x,y):
    # self.x = torch.tensor(x,dtype=torch.float32)
    # self.y = torch.tensor(y,dtype=torch.float32)
    input_shape = x.shape[1]
    self.x = x
    self.y = y.reshape(-1,1) 
    self.length = self.x.shape[0]

  def __getitem__(self,idx):
    x_raw = self.x
    x_tmp = self.x[idx, :]
    return torch.tensor(self.x[idx,:], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)

  def __len__(self): 
    return self.x.shape[0] 

In [19]:
trainset = WSDataset(x_data, y_data)
trainloader = DataLoader(trainset, batch_size=64, shuffle=False)

In [20]:
class Net(nn.Module):
  def __init__(self,input_shape):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_shape,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,1)
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.sigmoid(self.fc3(x))
    return x

In [21]:
learning_rate = 0.01
epochs = 700
# Model , Optimizer, Loss
input_shape=x.shape[1]
model = Net(input_shape=x.shape[1])
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [22]:
for x_sample, y_sample in trainloader:
    output = model(x_sample)
    break

In [23]:
def train_simple_network(model, loss_func, training_loader, epochs=20, device="cpu"): 
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)                    

    model.to(device)                                                             

    for epoch in tqdm(range(epochs), desc="Epoch"):                              
        model = model.train()                                                    
        running_loss = 0.0

        for inputs, labels in tqdm(training_loader, desc="Batch", leave=False):  
            inputs = moveTo(inputs, device)
            # labels = np.expand_dims(labels, axis=-1)                                      
            labels = moveTo(labels, device)                                      

            optimizer.zero_grad()                                                

            y_hat = model(inputs)
            
            shape_y_hat = y_hat.shape
            shape_labels = labels.shape
            
            # print(shape_y_hat)
            # print(shape_labels)

            loss = loss_func(y_hat, labels)                                      
            loss.backward()                                                      
            optimizer.step()                                                     
            running_loss += loss.item()    
        print(running_loss)

In [26]:
device = torch.device('cuda')
train_simple_network(model, loss_fn, trainloader, device=device, epochs=100)

Epoch:   1%|          | 1/100 [00:01<02:25,  1.47s/it]

0.19736407459867783


Epoch:   2%|▏         | 2/100 [00:01<01:11,  1.37it/s]

0.1952678913526062


Epoch:   3%|▎         | 3/100 [00:01<00:48,  2.02it/s]

...

0.10221620654717972


Epoch:  99%|█████████▉| 99/100 [00:29<00:00,  3.44it/s]

0.10165874545378206


Epoch: 100%|██████████| 100/100 [00:29<00:00,  3.40it/s]

0.1011314645112194
